# Line Search

A line search is used to find the distance along the descent direction of the next step of the optimization.  The scalar multiple $\alpha$ along the descent direction $d$ is found by minimizing the function below.

$$
\underset{\alpha}{\text{minimize}} f(x_k + \alpha d)
$$
where
* $f(...)$ is the function to minimize
* $x_k$ is the current solution
* $\alpha$ is a scalar 
* $d$ is a vector that describes the descent direction of the function

For first-order optimization problems the descent direction is given by the negative gradient $-\nabla f(x_k)$.

For second-order optimization problems the descent direction is given by by the product of the negative gradient and Hessian $-\nabla f(x_k) H_k$.

In [1]:
import numpy as np

## Line Search Method

In [2]:
def bracket_minimum(fx, x, s, k):
    """
    bracket_minimum returns interval [a,b] that brackets mininum of fx
    
    Parameters
    ----------
    fx : function
        function
    x : numpy.ndarray
        starting position around which bracket is found
    s : numpy.ndarray
        initial step size separating [a,b]
    k : float
        scaling factor applied to step size at each iteration

    Returns
    -------
    numpy.ndarray
        lower bound of bracket interval
    numpy.ndarray
        upper bound of bracket interval
    """
    a, fxa = x, fx(x)
    b, fxb = x + s, fx(x + s)
    if fxb > fxa:  # Invariant: a < b.
        a, b = b, a
        fxa, fxb = fxb, fxa
        s = -s
    while True:
        c, fxc = b + s, fx(b + s)
        if fxc > fxb:
            break
        a, fxa, b, fxb = b, fxb, c, fxc
        s = s * k
    if a < c:
        return a, c
    return c, a


def goldensection(fx, a, b, tol):
    """
    goldensection returns the minimum of fx over some interval [a,b]

    Parameters
    ----------
    fx : function
        function
    a : numpy.ndarray
        lower bound of interval that brackets minimum of fx
    b : numpy.ndarray
        upper bound of interval that brackets minimum of fx
    tol : float
        convergence threshold

    Returns
    -------
    numpy.ndarray
        point along interval [a,b] where fx is minimum
    """
    tau = (np.sqrt(5) - 1.) / 2.  # Golden ratio - 1.
    x1, x2 = a + (1. - tau) * (b - a), a + tau * (b - a)
    fx1, fx2 = fx(x1), fx(x2)
    
    while (b - a) > tol:
        if fx1 < fx2:
            b = x2
            # Treat x1 as the new x2.
            x2, fx2 = x1, fx1
            # Compute new x1.
            x1 = a + (1. - tau) * (b - a)
            fx1 = fx(x1)
        else:
            a = x1
            # Treat x2 as the new x1.
            x1, fx1 = x2, fx2
            # Compute new x2.
            x2 = a + tau * (b - a)
            fx2 = fx(x2)
    
    return x1


def line_search(fx, d, xk):
    """
    line_search returns the offset from xk where fx is minimum

    Parameters
    ----------
    fx : function
        function
    d : numpy.ndarray
        descent direction of fx, typically negative gradient at xk
    xk : numpy.ndarray
        starting position of search

    Returns
    -------
    alpha : float
        scalar multiple along descent direction where fx is minimum
    numpy.ndarray
        position xk where fx is minimum
    """

    # Objective function to minimize.
    fobj = lambda alpha: fx(xk + alpha * d)
    alpha0 = 1e-6

    # Find interval [a,b] closest to alpha0 that brackets the minimum.
    a, b = bracket_minimum(fobj, alpha0, s=1e-2, k=2.)

    # Find minimum within the bracket [a,b].
    alphak = goldensection(fobj, a, b, tol=1e-6)

    # Position where fx is minimum.
    xkmin  = xk + alphak * d
    
    return alphak, xkmin

Demonstrate the use of a line search to find the next step of the iteration when finding the minimmum of the function $f(x)$ when $x = (1,2,3)$ and $d=(0,-1,-1)$.

$$
f(x_1, x_2, x_3) = \sin(x_1 x_2) + \exp(x_2 + x_3) - x_3
$$

Problem is taken from example in section 4.2 of:
> Mykel J. Kochenderfer and Tim A. Wheeler. 2019. Algorithms for Optimization. The MIT Press.

In [3]:
fx = lambda x: np.sin(x[0]*x[1]) + np.exp(x[1]+x[2]) - x[2]
xk = np.array([1.,2.,3.])
d = np.array([0.,-1.,-1.])

alphak, xkmin = line_search(fx, d, xk)
print(alphak, xkmin)

np.testing.assert_almost_equal(alphak, 3.12705, decimal=5)
np.testing.assert_almost_equal(xkmin, np.array([1.,-1.126,-0.126]),
                               decimal=3)

3.1270455956291503 [ 1.        -1.1270456 -0.1270456]
